# Data Reporting and Communication - Geochemistry Example

This notebook is an example of going through the data report process to illustrate some aspects you may want to highlight (see the other pm notebook). This example data report is principally based on data which is relatively clean already - to highlight the key parts of the exercise without getting bogged down in the details of data munging which are often dataset specific.


## Read the Docs

As we're working with new datasets, new types of data, and different domains, you might want to put together an analysis or visualisation which we haven't yet encountered. We'd suggest that you check out the documentation pages for some of the key packages if you're after something specific, or you run into an error you can trace back to these libraries:
- [matplotlib](https://matplotlib.org/) for basic plotting (but allows control of many details where needed)
- [pandas](https://pandas.pydata.org) for data handling (our dataframe library)
- [seaborn](https://seaborn.pydata.org) for _nice_ data visualization
- [scipy](https://scipy.org) for scientific libraries (particularly `scipy.stats` which we'll use for fitting some more unusual probability distributions), and 
- [statsmodels](https://www.statsmodels.org/stable/index.html) which gives us some more expressive curve fitting approaches, should you wish to use them

## Import Your Dataset


This specific example is a dataset from my own research - it's an early version of a dataset I've put together to see whether we can effectively tell in which setting a rock formed based on its chemistry (tectonic setting classificaiton/discrimination).

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

I've added the dataset to Onedrive for this exercise - here we download it directly from there into memory:

In [ ]:
from fetch import get_onedrive_directlink

# here I've added a direct link to a pickled dataset stored in Onedrive
df = pd.read_pickle(get_onedrive_directlink('https://1drv.ms/u/s!As2ibEui13xmlv55DFaAn0HpuFVgdA?e=zjMif9'), compression='gzip')
df.drop(columns=df.columns[0], inplace=True) # an index snuck in here - we don't need it

In [ ]:
df.head()

In [ ]:
df.info()

---
## Why was this dataset collected/recorded?

* What would you like to do with it? Is it amendable to that use case?
* Does it have obvious limitations or restrictions to how it might be used?
* Is the data limited in relevance to a particular time period, area or site?
---

<em>
 
* This dataset was aggregated for the purpose of building a model to classify the setting in which different igneous rocks formed based on their geochemistry. 

* The tectonic settings are assigned based on geographical location - it is more complicated in some tectonic scenarios or where there might be overlap through space or in geological time.

* These rocks are all relatively recent (up to a few hundred million yeras old at most), and whether these samples are useful for classifying older rocks is uncertain.

* Whether these tectonic settings existed at all as you go further back in time is actually contentious - so the classificaiton problem isn't well formed under these scenarios.

* This dataset isn't likely to be directly useful to investigation of sedimentary rocks (although in some casees it could provide a some relevant info..), and may or may not be useful for looking at metamorphosed rocks

* Most of the samples are from land!

</em>

---
## Why might I be interested?

* What else might this be useful for?
* Could this be linked or integrated with another dataset?
* Could your solution to the problem be re-used in another area of the business?
---

<em>

* This relatively diverse dataset could be used for a variety of classificaiton and regression problems

* It would also be a handy reference for where you may want background values to provide geological context, or a basis for imputing 'missing' geochemical values for igneous rocks

* The geochemical variables in this dataset should be directly comparable to most other lithogeochemical datasets - and hence could be easily integrated. Some of the isotope ratios may need to be converted to different units or forms; some of the locations may also need to be converted.

* Only some of the samples have accurate positioning, and few have a 'depth' parameter (most are likely surface or near-surface samples rather than drilled).

* The classificaiton workflow developed for this problem can readily be adapted to many other geochemical and mineralogical classification problems.
    
</em>

---
## How big a dataset are we talking?

This one is relatively straightfoward, but provides some first-order constraints on what we may be able to do with it, and how you might want to work with your data:
* Number of records
* Number of variables
* Size on disk
---

In [ ]:
df.index.size

In [ ]:
df.columns.size

In [ ]:
df.info(memory_usage="deep")

---
* Are there multiple groups of records within your dataset (e.g. multiple sites, machines, instances or time periods)?
    * Is your target variable likely to be dependent on these groupings/is this key grouping your target variable (i.e. a classification problem)?
    * Are there similar numbers of records for each of these groups, or is it a bit imbalanced?
---

*My target variable is the principal grouping, which I've encoded as eight categories:*
* Back arc basins (BAB)
* Continental arcs (CA)
* Continental flood basalts (CF)
* Island arcs (IA)
* Intra-oceanic arcs (IOA)
* Mid-Ocean Ridge basalts (MOR)
* Ocean Island basalts (OI)
* Oceanic plateuas (OP)
*

In [ ]:
df.Srcidx.unique()

*This is a reasonably imbalanced dataset!*

In [ ]:
df.Srcidx.value_counts()

---
* Is the dataset in a tidy format? How might you need to rearrange it if not? 
    1. Each variable you measure should be in one column.
    2. Each different observation of that variable should be in a different row.
 
<!-- <div class='alert alert-success'> -->
<b>Note:</b> the last two tidy data points about separate tables are excluded here; for most ML applications you'll need a single fully-integrated table. Depending on your dataset, it may make sense to keep individual table separate until after data processing, however.
<!-- </div> -->
---

*This dataset is already quite tidy for a single-table dataset (although this took some munging...!)*

---
* If your records relate to individual measurements or samples, are they regularly spaced (in time and/or space)? What's the frequency of the records?
---

*These samples are unevenly distributed across the globe; some parts of the world might not have much data (although not all samples have point locations!):*

In [ ]:
ax = df.plot.scatter('Longitude', 'Latitude', color='g')
ax.scatter(df[['Longitudemin', 'Longitudemax']].mean(axis=1), df[['Latitudemin', 'Latitudemax']].mean(axis=1), color='purple')

---
## What are the variables?

Provide an overview of the types and groupings of variables, where relevant:
* What are the variable names? Should you rename these for clarity?
---

<em>
    
* Some of the variables have overlap (espeically for metadata) - and some of the names could be updated for clarity (e.g. 'Srcidx' isn't very informative!).
* The geochemical data is labelled in a straightfoward way.
* The variables L0-L3 are parameterisations of geochemistry and could be more verbosely named.
    
</em>

In [ ]:
df.columns.tolist()#[:20]

---

* Which variables are your targets (what you want to predict) and which are your likely inputs (what you'll use to predict your target)?

---

<em>

* Here the target is the tectonic setting category in <b>Srcidx</b>, and most of the geochemical data variables are vaiable inputs.
* The metadata may not be particularly useful in this case, especially given that it's not univeral across all samples.
    
</em>

---
* How have the variables been measured/recorded?
* Are units are important? Is the entire table in a consistent format/set of units?
---

<em>
    
* The geochemical data have been measured using a variety of techniques (and to some degree the individual records represent multiple analyses).
* Geochemical data recorded as elements (e.g. 'Cr') are in units of ppm; data recorded as oxides (e.g. 'Cr2O3') are in units of Wt%.
    
</em>

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(8, 4))

bins = np.linspace(0, 1000, 100)
df['Cr'].plot.hist(bins=bins, ax=ax[0])
df['Cr2O3'].plot.hist(bins=bins/10000, ax=ax[1]) # wt%/ppm = 10000 - this should be a comparable scale!

ax[0].set(xlabel='Cr (ppm)')
ax[1].set(xlabel='Cr2O3 (Wt%)')
plt.tight_layout()

---
* Are variables in the right formats?
    * Have some numerical variables been converted to strings/objects?
    * Are dates recorded in a convenient format?
    * Do you have [categorical](https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html) variables which you could appropriately encode?
---

*Data are largely in the correct formats - although geological ages aren't here converted to dates....*

In [ ]:
df.dtypes.head()

---
* Are some data missing?
    * Are they randomly or systemtically missing?
    * Is there a correlation between 'missingness' across variables?
    * How is missing data recorded? Are there more than one form of missing data, and if so do you want to retain that information (e.g. 'below detection', 'not measured')?
    * What are your options for [dealing with the missing data](https://pandas.pydata.org/pandas-docs/stable/user_guide/missing_data.html)? Do you need to drop these rows, or can you fill the values/impute the values?
---

<em>

* We have a number of columns which are missing most of the data
    
* In some cases data seem to be missing at random - but as we have compositional data it's partially missing due to being below a threshold for detection
 
* In many cases elements were simply not measured - which is partially due to methods, and partly due to choices for different rocks, so is not independent of our target variable!
  
* Missing data is here replaced with np.nan

* We could impute some missing data, but given the nature of the data it may be better to select a low-missingness subset of features...

</em>

In [ ]:
percentage_present = (df.isnull().sum(axis=0) / df.index.size * 100)

In [ ]:
percentage_present.to_frame().head(20).style.background_gradient(cmap='viridis_r')

---
* How are the variables distributed?
    * Are they approximately normally distributed?
    * Will you need to transform these before using them in a machine learning pipeline?
    * What are appropriate values for your target variable (i.e. continuous real values, continous positive values, boolean, categories)? 
---

<em>

* Geochemical data are not expected to be normally distributed - instead they're likely to approximately log-normally distributed (or more specifically ratios of geochemical variables are expected to be..)
    
* As such some transformation will likely be needed.
    
* In this case the target variable is categorical, so is well bounded - we don't need to worry about continuous values here!
    
* We might have some precision issues (e.g. see Cr2O3)
    
* We will have some 'below detection' truncation
    
* Data for the same element will be distributed differently if it's recorded as an oxide or element (e.g. Cr vs Cr2O3; partly due to methods and relative detection limits)
    
</em>

In [ ]:
ax = df.Cr2O3.plot.hist(bins=bins/10000)
ax.vlines(np.arange(0, 0.11, 0.01), ymin=190, ymax=200, color='k') # add some lines to illustrate where precision might be interfering

In [ ]:
fig, ax = plt.subplots(1, 2, sharey=True)
df.Cr.plot.hist(bins=bins, color='0.5', ax=ax[0])

logbins = np.linspace(np.log(bins[1]), np.log(bins[-1]), 100)
df.Cr.apply(np.log).plot.hist(bins=logbins, color='0.5', ax=ax[1]) # compare log-scaled data over the same range

In [ ]:
ax = df['Th'].plot.hist(bins=bins/100)
ax.vlines(np.arange(0, 10, 1), ymin=1400, ymax=1500, color='k') # add some lines to illustrate where precision might be interfering

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 4))
df['Ti'].plot.hist(bins=100, ax=ax[0])
df['TiO2'].plot.hist(bins=np.linspace(0, 6, 100), ax=ax[1])
ax[0].set(xlabel='Ti (ppm)')
ax[0].set(xlabel='TiO2 (wt%)')
plt.tight_layout()

---
* What do the correlations of variables look like? Are there 'blocks' or groups of variables which are correlated with one another, or is each providing different information?
---

<em>

* There are some strong correlations and 'blocks' within the dataset - one of these is the Rare Earth Elements!

* There is some variety in the structure - different elements seem to be providing independent information.

* Some parameters are either perfectly or nearly-perfectly correlated (e.g. ratios and normalised ratios, location min-max pairs)

</em>

In [ ]:
corr = df.corr()
f, ax = plt.subplots(figsize=(14, 14))
sns.heatmap(corr, square=True, linewidths=0.01, cbar_kws={"shrink": .8}, cmap='viridis')

---
* Are there outliers?
    * Are they related to incorrect data, rare events or potential data entry issues?
    * Are they likely to have a negative impact on your model, or are they an inherent feature of the dataset?
    * If you're to remove them, what's a good way of selecting them?
---

*Perhaps - but distinguishing 'outliers' from strange rocks might not be something I can confidently do based on the data alone (and a single sample of each rock..). For this reason I'll leave them in for the time being - unless I can establish clear data-driven reasons for their exclusion.*

---
## Visualising Key Relationships


* What are some key relationships within your dataset?
---

*A key visualisation which is used for the graphical equivalent of this task is the 'Pearce plot' - a plot of Th/Yb over Nb/Yb (illustrating reasonable separation for just two/three dimensions..):*

In [ ]:
fig, ax = plt.subplots(1, figsize=(8, 6))

variables = ['Nb/Yb', 'Th/Yb']
ax.set(yscale='log', xscale='log', xlabel=variables[0], ylabel=variables[1])

for setting, subdf in df.groupby('Srcidx'):
    ax.scatter(*subdf[variables].T.values, label=setting, alpha=0.8/np.log(subdf.index.size))

# add a legend and alter the opacity so we can see what's what
leg = ax.legend(frameon=False, facecolor=None, bbox_to_anchor=(1,1))
for lh in leg.legendHandles: 
    lh.set_alpha(1)

---
* How might you investigate this dataset further?
---

---
* Do you expect any major hurdles for getting this dataset analysis ready? Are there any key decisions you need to make about pre-processing?
---

<em>

* Dealing with missing data

* Identifying subtle errors in data entry/units
    
* Dealing with imprecision issues
    
* Identifying poor quality analyses
    
* Spatial coverage, biases and validity
    
* & more... 
    
</em>

---
## Optional: Find another dataset that we could fuse with this one.

* Are there other datasets which might provide some additional context to solve your problem (e.g. bringing in data from logs, weather data, imagery)?
---

---
* Could your dataset be integrated with data from further along the processing chain/another part of the business to solve problems there?
---